# Practice 4: Extractive Question Answering

This notebook provides a tutorial how to 

In [1]:
%%capture
!pip install datasets transformers torch

In [ ]:
import datasets

dataset = datasets.load_dataset("FIdo-AI/ua-squad", split="train")
dataset

/home/robinhad/Projects/ucu-nlp-course/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Repo card metadata block was not found. Setting CardData to empty.


Dataset({
    features: ['version', 'data'],
    num_rows: 13859
})

In [3]:
def extract_columns(example):
    return {
        "answer": example["data"]["Answer"],
        "context": example["data"]["Context"],
        "question": example["data"]["Question"],
    }

dataset = dataset.map(extract_columns)
dataset = dataset.remove_columns(["version", "data"])
dataset[0]

{'answer': 'виготовлення продуктів та поширення досвіду, які люди хочуть отримати й можуть собі дозволити',
 'context': '5 січня 2012 року Вест оголосив про створення компанії для творчого контенту DONDA, названої на честь його покійної матері Донди Вест. Під час представлення Вест заявив, що компанія "продовжить там, де зупинився Стів Джобс"; DONDA діятиме як "дизайнерська компанія, яка забезпечить мислителям творчий простір для реалізації своїх мрій та ідей" з "метою виготовлення продуктів та поширення досвіду, які люди хочуть отримати й можуть собі дозволити". Вест, як відомо, мало говорить про діяльність компанії, відсутні як офіційний веб-сайт, так і представлення в соціальних мережах. Креативна філософія DONDA містить необхідність "розміщувати творців у спільному просторі разом із подібними думками", щоб "спростити та естетично вдосконалити все, що ми бачимо, смакуємо, торкаємось та відчуваємо". Сучасні критики відзначають незмінну мінімалістичну естетику, яка повторюється в бага

In [4]:
ukr_squad = dataset.filter(lambda x: len(x["answer"]) > 0)
dataset

Dataset({
    features: ['answer', 'context', 'question'],
    num_rows: 13859
})

In [5]:
ukr_squad = ukr_squad.train_test_split(0.1, seed=42) # split at 10%
ukr_squad

DatasetDict({
    train: Dataset({
        features: ['answer', 'context', 'question'],
        num_rows: 9838
    })
    test: Dataset({
        features: ['answer', 'context', 'question'],
        num_rows: 1094
    })
})

In [6]:
ukr_squad["train"][1]

{'answer': 'кооперативного полювання',
 'context': 'Антропологи вважають, що найважливішою перевагою собак, було використання їхнього гарного нюху в мисливстві. Взаємозв’язок між присутністю собаки та успіхом на полюванні часто згадується, як основна причина одомашнення вовка. У 2004 році було проведено дослідження груп мисливців із собаками та без них. Результат кількісно підтримують гіпотезу про те, що перевага кооперативного полювання була важливим чинником приручення вовків.',
 'question': 'Як називають полювання, під час якого люди та собаки діють разом?'}

# Select the best model
https://huggingface.co/spaces/mteb/leaderboard


In [7]:
from transformers import AutoTokenizer

model_name = "panalexeu/xlm-roberta-ua-distilled"

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:
def preprocess_function(examples):
    #print(examples)
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answer"]
    contexts = examples["context"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        
        if answer is None:
            print(questions[i], answer, contexts[i])
        start_char = contexts[i].find(answer)
        end_char = start_char + len(answer)
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [9]:
tokenized_ukr_squad = ukr_squad.map(preprocess_function, batched=True, remove_columns=ukr_squad["train"].column_names)

In [10]:
tokenized_ukr_squad

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 9838
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 1094
    })
})

In [23]:
tokenizer.decode(tokenized_ukr_squad["train"][0]["input_ids"])

'<s> Який коледж був попередником інженерного факультету Токійського університету?</s></s> У Японії технологічний інститут (工業 大学, kōgyō daigaku?) є типом університету, який спеціалізується на науках. Імператорський інженерний коледж, до речі, також був попередником інженерного факультету Токійського університету.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [11]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [12]:
from transformers import AutoModelForQuestionAnswering, GPT2LMHeadModel, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_name)

Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at panalexeu/xlm-roberta-ua-distilled and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    weight_decay=0.01,
    save_total_limit=3,
    save_strategy="epoch",
    eval_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    torch_compile=True,
    #fp16=True,
    #gradient_checkpointing=True,
    #gradient_accumulation_steps=4,
    #report_to="tensorboard"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ukr_squad["train"],
    eval_dataset=tokenized_ukr_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/tmp/ipykernel_13790/2327706108.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,2.722400,1.513587
2,1.421500,1.287809
3,1.090000,1.204214
4,0.874900,1.230519
5,0.735100,1.251971
6,0.646100,1.276942


W1006 15:40:02.764000 13790 torch/fx/experimental/symbolic_shapes.py:6823] [0/1] _maybe_guard_rel() was called on non-relation expression Eq(s43, 1) | Eq(s72, s43)


TrainOutput(global_step=1848, training_loss=1.248330698384867, metrics={'train_runtime': 382.0846, 'train_samples_per_second': 154.489, 'train_steps_per_second': 4.837, 'total_flos': 1.1567868717459456e+16, 'train_loss': 1.248330698384867, 'epoch': 6.0})

In [14]:
trainer.create_model_card()

In [24]:
from transformers import pipeline

qa_model = pipeline("question-answering", model=model.to("cpu"), tokenizer=tokenizer)
question = "що відправлять для ЗСУ?"
context = "Про це повідомив міністр оборони Арвідас Анушаускас. Уряд Литви не має наміру зупинятися у військово-технічній допомозі Україні. Збройні сили отримають антидрони, тепловізори та ударний безпілотник. «Незабаром Литва передасть Україні не лише обіцяні бронетехніку, вантажівки та позашляховики, але також нову партію антидронів та тепловізорів. І, звичайно, Байрактар, який придбають на зібрані литовцями гроші», - написав глава Міноборони."
qa_model(question = question, context = context)

Device set to use cuda:0


{'score': 0.07516611367464066,
 'start': 152,
 'end': 198,
 'answer': 'антидрони, тепловізори та ударний безпілотник.'}

In [30]:
question = "хто такий Андрій Супрун?"
context = """Передумови для розвитку локального ринку капіталу в Україні існують, тому з об’єднанням експертних та політичних зусиль такий ринок може бути створений.
Так вважає начальник управління корпоративних прав та депозитарної діяльності Національного банку України Андрій Супрун, передає "Інтерфакс-Україна".
"Крок за кроком, я впевнений, що ми зможемо побудувати як локальний ринок, достатньо цікавий, привабливий, так і якісно інтегрувати його в міжнародні фінанси", – сказав він.
На думку Супруна, щодо наявності на ринку інвесторів зараз ситуація значно краща, ніж 5-10 років тому.
Як аргумент він навів успішний довоєнний досвід заходження на український ринок державних облігацій іноземних інвесторів через створений "лінк" із Clearstream із залученням "мільярдів валютних коштів".
"Цей ринок міжнародних інвестицій величезний, а ми на ньому просто крапелька маленька, яку ніхто не помічає. Навіть якщо ми ще декілька крапельок сюди залучимо, ми отримаємо тут дуже великий мультиплікатор", – зазначив представник Нацбанку. """
qa_model(question = question, context = context)

{'score': 0.6673192977905273,
 'start': 164,
 'end': 258,
 'answer': 'начальник управління корпоративних прав та депозитарної діяльності Національного банку України'}